Importación de librerías

In [3]:
import pandas as pd
from Dataframes import contratos_df, pagos_df
import os

c:\Econda\Proyecto-Data-Science\Dataframes.py:31: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  pagos_df = pd.read_csv(os.path.join('data', 'pagos.csv'))


Carga de datos

In [4]:
data_dir = './data/'

proyectos_df = pd.read_csv(os.path.join(data_dir, 'proyectos.csv'))
# contratos_df = pd.read_csv(os.path.join(data_dir, 'contratos.csv'))
# pagos_df = pd.read_csv(os.path.join(data_dir, 'pagos.csv'))
ofertas_df = pd.read_csv(os.path.join(data_dir, 'ofertas.csv'))

Porcentaje de atraso de las obras públicas

In [5]:
contratos_df['ATRASO'] = (
    pd.to_datetime(contratos_df['FECHATERMINO']) - 
    pd.to_datetime(contratos_df['FECHATERMINOORIGINAL'])
).dt.days

obras_con_atraso = contratos_df[contratos_df['ATRASO'] > 0].shape[0]
total_obras = contratos_df.shape[0]
porcentaje_atraso = (obras_con_atraso / total_obras) * 100

print(f"Porcentaje de obras con atraso: {porcentaje_atraso:.2f}%")

Porcentaje de obras con atraso: 46.34%


Tipo de obra que tarda más en realizarse

In [6]:
contratos_df['DURACION'] = (
    pd.to_datetime(contratos_df['FECHATERMINO']) - 
    pd.to_datetime(contratos_df['FECHAINICIO'])
).dt.days

tipo_obra_mas_lenta = contratos_df.groupby('SERVICIO', observed=False)['DURACION'].mean().idxmax()

print(f"Tipo de obra que tarda más en realizarse: {tipo_obra_mas_lenta}")

Tipo de obra que tarda más en realizarse: VIALIDAD


Tipo de obra más cara

In [7]:
tipo_obra_mas_cara = contratos_df.groupby('SERVICIO', observed=False)['MONTOVIGENTE'].mean().idxmax()

print(f"Tipo de obra más cara: {tipo_obra_mas_cara}")

Tipo de obra más cara: VIALIDAD


Variación del presupuesto por año

In [8]:
pagos_df['FECHAPAGO'] = pd.to_datetime(pagos_df['FECHAPAGO'])
pagos_df['ANIO'] = pagos_df['FECHAPAGO'].dt.year

presupuesto_anual = pagos_df.groupby('ANIO')['TOTALPAGADO'].sum()

#Convertimos esto a un dataframe

presupuesto = {}

presupuesto['Año'] = presupuesto_anual.index
presupuesto['Presupuesto'] = presupuesto_anual.values

presupuesto = pd.DataFrame(presupuesto)

#Datos de la inflación son sacados de https://www.datosmundial.com/america/chile/inflacion.php

inflacion_dolares = {
    2023: 1.00,  # Valor base
    2022: 800 / 870,  # Valor dólar 2023 / Valor dólar 2022
    2021: 800 / 825.18,  # Valor dólar 2023 / Valor dólar 2021
    2020: 800 / 780.52,  # Valor dólar 2023 / Valor dólar 2020
    2019: 800 / 670.62,  # Valor dólar 2023 / Valor dólar 2019
    2018: 800 / 630.22,  # Valor dólar 2023 / Valor dólar 2018
    2017: 800 / 635.73,  # Valor dólar 2023 / Valor dólar 2017
    2016: 800 / 670.28,  # Valor dólar 2023 / Valor dólar 2016
    2015: 800 / 635.14,  # Valor dólar 2023 / Valor dólar 2015
    2014: 800 / 528.90,  # Valor dólar 2023 / Valor dólar 2014
    2013: 800 / 469.82,  # Valor dólar 2023 / Valor dólar 2013
    2012: 800 / 467.18,  # Valor dólar 2023 / Valor dólar 2012
    2011: 800 / 468.25,  # Valor dólar 2023 / Valor dólar 2011
    2010: 800 / 502.45   # Valor dólar 2023 / Valor dólar 2010
}

#aquí básicamente convierto todos los presupuestos de años anteriores a lo que valdría ese presupuesto hoy 

presupuesto= presupuesto.drop(14)

def poner_puntos(numero): # le doy los puntitos bonitos
    return "{:,}".format(numero).replace(",", ".")

presupuesto['variacion'] = 'N/a'

for x in range(len(presupuesto)):

    numero = round(presupuesto.loc[x, 'Presupuesto'] * inflacion_dolares[int(presupuesto.loc[x, 'Año'])])

    presupuesto.loc[x, 'Presupuesto'] = numero

    if x > 0:

        presupuesto.loc[x, 'variacion'] = poner_puntos(round(numero - presupuesto.loc[x - 1, 'Presupuesto']))

    else:

        presupuesto.loc[x, 'variacion'] = 'N/a'

presupuesto['Presupuesto'] = presupuesto['Presupuesto'].apply(poner_puntos)

presupuesto

,Año,Presupuesto,variacion
0,2010,395.724.631.421,N/a
1,2011,992.300.554.398,596.575.922.977
2,2012,1.363.740.084.642,371.439.530.244
3,2013,1.598.184.320.986,234.444.236.344
4,2014,1.485.506.431.230,-112.677.889.756
5,2015,1.401.559.999.417,-83.946.431.813
6,2016,1.250.496.109.800,-151.063.889.617
7,2017,1.341.498.933.898,91.002.824.098
8,2018,1.298.223.013.282,-43.275.920.616
9,2019,1.448.423.658.869,150.200.645.587


Empresa más contratada

In [10]:
empresa_mas_contratada = pagos_df['NOMBREADJUDICADO'].value_counts().idxmax()

print(f"Empresa más contratada: {empresa_mas_contratada}")

Empresa más contratada: CONST. DE PAVIMENTOS ASFALTICOS BITUMIX S.A.


Promedio de atraso de las obras publicas

In [11]:
promedio_atraso = contratos_df[contratos_df.ATRASO > 0]['ATRASO'].mean()
print(f'El promedio de atraso de las obras publicas es de {round(promedio_atraso, 2)} diasa')

El promedio de atraso de las obras publicas es de 102.77 diasa
